<a href="https://colab.research.google.com/github/Takouabelwafa/Test-Technique-Full-Pack-AI-/blob/main/Projet4_TestTechnique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Projet 4 — OCR intelligent pour lecture automatique de factures
**Objectif :** Extraire texte structuré (nom, montant, date) à partir d'une image PDF ou JPEG                                                                                        
**Techniques **: Tesseract OCR, OpenCV, post-processing NLP                     
**📦 Dataset :** Invoice OCR Dataset                                          
**🔧 Tâches :**
* Extraction texte avec Tesseract
* Nettoyage regex
* Classification des champs (entité nommée)
* Export JSON structuré

In [ ]:
#Préparer l'environement

In [ ]:
pip install pytesseract opencv-python pandas regex


In [ ]:
import pytesseract
print(pytesseract.get_tesseract_version())

4.1.1


In [ ]:
# Pour convertir les PDF en images
!apt-get install -y poppler-utils

# Bibliothèques Python
!pip install pdf2image pytesseract opencv-python-headless pandas

# Pour l'export JSON
import json


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.11).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


**Charger le PDF ou les images**

*  Pour les PDFs, tu peux convertir chaque page en image avec pdf2image :

In [ ]:
from google.colab import files
from pdf2image import convert_from_path

# Upload du fichier PDF
uploaded = files.upload()  # Choisir un fichier dans notre cas 'facture.pdf'

# Conversion PDF  to images (300 dpi)
pages = convert_from_path('facture.pdf', 300)

# Sauvegarder la première page comme exemple
pages[0].save('page1.jpg', 'JPEG')


Saving facture.pdf to facture.pdf


# ** Extraction du texte avec Tesseract OCR **

In [ ]:
# Installer Tesseract
!apt-get install -y tesseract-ocr

# Installer le pack de langue française
!apt-get install -y tesseract-ocr-fra


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-fra
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 527 kB of archives.
After this operation, 1,145 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-fra all 1:4.00~git30-7274cfa-1.1 [527 kB]
Fetched 527 kB in 1s (635 kB/s)
Selecting previously unselected package tesseract-ocr-fra.
(Reading database ... 126748 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-fra_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-fra (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-fr

In [ ]:
!tesseract --list-langs


List of available languages (3):
eng
fra
osd


In [ ]:
import pytesseract
import cv2
from PIL import Image
import numpy as np

# Charger l'image
image = cv2.imread('page1.jpg')
# Conversion en niveaux de gris
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# OCR avec Tesseract
text = pytesseract.image_to_string(gray, lang='fra')  # 'fra' pour français
print(text)


DA INV24.FR

Destinataire:

[Nom de l'entreprise du client]

[Adresse de l'entreprise du client]
SIRET: [SIRET du client]

No de TVA: [Numéro de TVA du client]

Désignation
mon produit
mon service

[Nom de l'entreprise]
SIRET: [SIRET]
No de TVA: [Numéro de TVA]

Facture No 552

Quantité

2
3

[Adresse]
Tél: [Téléphone]

Date de facturation: 12/07/2021
Echéance: 26/07/2021

Prix Total
50,00 100,00
100,00 300,00

Total: 400,00 €
TVA(20%): 80,00 €
Total TTC: 480,00 €

[Nom de la banque]
[Compte bancaire (IBAN)]
SWIFT/BIC: [SWIFT/BIC]



# Nettoyage du texte avec Regex

In [ ]:
import re

# Extraire la date (format classique dd/mm/yyyy)
date_match = re.search(r'\b\d{2}/\d{2}/\d{4}\b', text)
date = date_match.group() if date_match else None

# Extraire le montant (exemple : 1234.56 ou 1 234,56)
amount_match = re.search(r'\b\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d{2})\b', text)
amount = amount_match.group() if amount_match else None

# Extraire un nom/fournisseur (exemple simple : première ligne du texte)
lines = text.split('\n')
supplier = lines[0] if lines else None

print("Date :", date)
print("Montant :", amount)
print("Fournisseur :", supplier)


Date : 12/07/2021
Montant : 50,00
Fournisseur : DA INV24.FR


# Classification des champs (option NLP avancé)

In [ ]:
!pip install spacy
import spacy

nlp = spacy.blank("fr")
doc = nlp(text)

# Ici tu pourrais créer des entités personnalisées : DATE, MONTANT, NOM
for ent in doc.ents:
    print(ent.text, ent.label_)


# Export en JSON structuré

In [ ]:
invoice_data = {
    "fournisseur": supplier,
    "date": date,
    "montant": amount
}

# Sauvegarde JSON
with open("facture.json", "w", encoding="utf-8") as f:
    json.dump(invoice_data, f, ensure_ascii=False, indent=4)

# Afficher pour vérification
print(json.dumps(invoice_data, ensure_ascii=False, indent=4))


{
    "fournisseur": "DA INV24.FR",
    "date": "12/07/2021",
    "montant": "50,00"
}


# Automatisation pour plusieurs pages ou fichiers

In [ ]:
all_invoices = []

for i, page in enumerate(pages):
    page.save(f'page{i}.jpg', 'JPEG')
    img = cv2.imread(f'page{i}.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang='fra')

    # Extraction regex
    date_match = re.search(r'\b\d{2}/\d{2}/\d{4}\b', text)
    amount_match = re.search(r'\b\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d{2})\b', text)
    lines = text.split('\n')
    supplier = lines[0] if lines else None

    all_invoices.append({
        "fournisseur": supplier,
        "date": date_match.group() if date_match else None,
        "montant": amount_match.group() if amount_match else None
    })

# Export final
with open("invoices.json", "w", encoding="utf-8") as f:
    json.dump(all_invoices, f, ensure_ascii=False, indent=4)
